# 生成数据集

In [1]:
import torch
import numpy as np

In [40]:
num_inputs=2
num_examples=1000
true_w=[2,-3.4]
true_b=4.2
features=torch.tensor(np.random.normal(0,1,(num_examples,num_inputs)),dtype=torch.float)
labels=true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels+=torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float)

# 读取数据

In [41]:
import torch.utils.data as Data

batch_size=10
#将训练数据的特征与标签组合
dataset=Data.TensorDataset(features,labels)
#随机读取小批量
data_iter=Data.DataLoader(dataset,batch_size,shuffle=True)

In [6]:
for X,y in data_iter:
    print(X,y)
    break

tensor([[ 0.9597, -0.8151],
        [-0.7712, -1.7511],
        [-1.4989,  0.2438],
        [ 0.7129, -1.3008],
        [ 2.0205, -0.7493],
        [-1.4020,  0.8971],
        [ 0.4768,  2.4293],
        [-1.4038,  1.6459],
        [ 1.0127,  1.2485],
        [-0.5808,  1.0980]]) tensor([ 8.8744,  8.6144,  0.3904, 10.0347, 10.7967, -1.6730, -3.1013, -4.2095,
         1.9659, -0.6816])


# 定义模型

In [30]:
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear=nn.Linear(n_feature,1)
    def forward(self,x):
        y=self.linear(x)
        return y
net=LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [14]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.5591, -0.4127]], requires_grad=True)
Parameter containing:
tensor([0.4805], requires_grad=True)


In [42]:
#写法一
net=nn.Sequential(
    nn.Linear(num_inputs,1)
    )
#写法二
net=nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))
#写法三
from collections import OrderedDict
net=nn.Sequential(OrderedDict([
    ('linear',nn.Linear(num_inputs,1))
    ]))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


# 初始化模型参数

In [43]:
from torch.nn import init
init.normal_(net[0].weight,mean=0,std=0.01)
init.constant_(net[0].bias,val=0)

Parameter containing:
tensor([0.], requires_grad=True)

# 定义损失函数

In [44]:
loss=nn.MSELoss()

# 定义优化算法

In [45]:
import torch.optim as optim

optimizer=optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [36]:
optimizer=optim.SGD([
    {'params':net.subnet1.parameters()},
    {'params':net.subnet2.parameters(),'lr':0.01}
],lr=0.03)

AttributeError: 'Sequential' object has no attribute 'subnet1'

In [24]:
for param_group in optimizer.param_groups:
    param_group['lr']*=0.1

In [25]:
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    momentum: 0
    nesterov: False
    weight_decay: 0
)


# 训练模型

In [46]:
num_epochs=3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        output=net(X)
        l=loss(output,y.view(-1,1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d,loss:%f'%(epoch,l.item()))

epoch 1,loss:0.000382
epoch 2,loss:0.000088
epoch 3,loss:0.000167


In [47]:
dense=net[0]
print(true_w,dense.weight)
print(true_b,dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 2.0004, -3.3993]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2001], requires_grad=True)
